# Age-Detection

## Environment

### System

- Os: Windows 11 64bit
- Python: 3.11.9

### Package

- pytorch: cu126 
- pillow: 11.1.0
- lightgbm: 4.6.0
- pandas: 2.2.3
- matplotlib: 3.10.1
- ipykernel: 6.29.5
- torchinfo: 1.8.0

## Impelementation

### Import libraries

In [ ]:
import scripts.augment as sa
import scripts.models as sm
import scripts.preprocessing as sp
import scripts.backbone as b
import scripts.regressor as reg
import scripts.evaluate as ev

### Define constant

In [ ]:
# Define paths
wiki_mat_path = "./data/wiki/wiki.mat"
verified_image_path = "./data/verified_image/"
processed_images_catch_path = "./data/processed/"
processed_metadata_catch_path = "./data/processed/processed_data.csv"
augmented_images_catch_path = "./data/augmented/"
augmented_metadata_catch_path = "./data/augmented/augmented_data.csv"

# Define variables
min_age = 0
max_age = 100
max_samples = 500

### Data preprocessing

In [ ]:
# Process Wiki metadata
wiki_metadata = sp.process_wiki_metadata(
    wiki_mat_path, 
    min_age=min_age, 
    max_age=max_age, 
    verified_image_path=verified_image_path,
    use_cached=True,
    cache_path=processed_metadata_catch_path
)

### Data augmentation

In [ ]:
wiki_metadata = sa.augment_facial_dataset(
    input_csv_path=processed_metadata_catch_path,
    input_base_path=processed_images_catch_path,
    output_base_path=augmented_images_catch_path,
    output_csv_path=augmented_metadata_catch_path,
    max_samples_per_age=max_samples,
    use_cached=True,
    cache_path=augmented_metadata_catch_path
)

### Load models

In [ ]:
# Load all models
device = sm.get_device()
models = sm.load_all_models(pretrained=True, train_mode=True, device=device)

# Print model summaries
sm.print_model_summaries(models)

### Fine tuning models

In [ ]:
data_loader = ev.load_age_data(augmented_metadata_catch_path)

fine_tuned_efficient_net_b0 = b.FineTunedBackbone(models['efficientnet_b0'])
fine_tuned_efficient_net_b0.finetune(data_loader['train_loader'], data_loader['val_loader'], epochs=5)

### LightGBM model

In [ ]:
model = reg.EfficientLightGBM(fine_tuned_model=fine_tuned_efficient_net_b0)
model.train_gbm(data_loader['train_loader'])

### Save the model

In [ ]:
model.save_model('./models/efficient-light-gbm.pkl')

new_model = reg.EfficientLightGBM()
new_model.load_model("./models/efficient-light-gbm.pkl")

In [ ]:
#model = reg.EfficientLightGBM()
#model.load_model('./models/efficient-light-gbm.pkl')

# 加载数据
data_loader = ev.load_age_data(
    metadata_path='./data/augmented/augmented_data.csv',
    use_cached=True
)

# 模型评估（会自动出图）
results = ev.evaluate_efficient_lgbm(model, data_loader['test_loader'])

# 打印结果
for k, v in results.items():
    print(f"{k}: {v:.4f}")